In [4]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import yfinance as yf
from pandas_datareader import data as web
import os 
import pickle
import numpy as np
from collections import Counter
from nsepy import get_history
from finta import TA
import copy
from tqdm import tqdm


style.use('ggplot')
#%matplotlib qt

In [5]:
df = get_history(symbol="HDFCBANK", start=dt.date(2019,1,1), end = dt.date.today())
end_date = df.index[-1]
ondate = end_date
df_extracted = pd.read_csv('stocks_dfs/{}.csv'.format("HDFCBANK"), header= 0)
df_extracted.index = df_extracted.Date
extracted_date = dt.datetime.strptime(df_extracted.index[-1] , '%Y-%m-%d').date() +  dt.timedelta(days=1)
print(end_date, ondate)

2020-08-24 2020-08-24


In [6]:
complete_results = {} 
company_template = {'REMARKS': "", 'BREAKPOINTS' : "", 'strat_incdec':{'0-200' : 0, '0-100': 0, '8-21': 0, '50-200': 0},  'strat_smaema' : 0 , 'strat_point5all': {'above_close' : 0, 'below_close':0}, 'strat_macd' : {'normal_crossover' : 0, 'middle_crossover' : 0, 'hist_turnover': 0}, 'strat_hieken_ashi' :{'Bullish': 0, 'Bearish': 0}, 'strat_fisher': {'Bearish' : 0, 'Bullish': 0, 'Middle_cross': 0}, 'strat_stochrsi': {'Bullish' : 0, 'Bearish': 0, 'Middle_cross': 0} }

In [7]:
def get_index(ticker, ondate):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    today_index = df.index[df.Date == str(ondate)].tolist()
    if len(today_index) == 0:
        return 0, 0, 0
    today_index = today_index[0]
    yesterday_index = today_index-1
    yesterday2_index = today_index-2 
    return (today_index, yesterday_index, yesterday2_index)

In [8]:
def generate_data(df):
    df['3ma'] = df['Close'].rolling(window = 3, min_periods = 0).mean()
    df['5ma'] = df['Close'].rolling(window = 5, min_periods = 0).mean()
    df['35d'] = ((df['3ma']-df['5ma'])/df['3ma'])*100;
    df['8ma'] = df['Close'].rolling(window = 8, min_periods = 0).mean()
    df['58d'] = ((df['5ma']-df['8ma'])/df['5ma'])*100;
    df['14ma'] = df['Close'].rolling(window = 14, min_periods = 0).mean()
    df['814d'] = ((df['8ma']-df['14ma'])/df['8ma'])*100;
    df['21ma'] = df['Close'].rolling(window = 21, min_periods = 0).mean()
    df['1421d'] = ((df['14ma']-df['21ma'])/df['14ma'])*100;
    df['50ma'] = df['Close'].rolling(window = 50, min_periods = 0).mean()
    df['2150d'] = ((df['21ma']-df['50ma'])/df['21ma'])*100;
    df['100ma'] = df['Close'].rolling(window = 100, min_periods = 0).mean()
    df['50100d'] = ((df['50ma']-df['100ma'])/df['50ma'])*100;
    df['200ma'] = df['Close'].rolling(window = 200, min_periods = 0).mean()
    df['100200d'] = ((df['100ma']-df['200ma'])/df['100ma'])*100;    
    df['ema10'] = df['Close'].ewm(span=10).mean()
    #display(df.tail())
    return df

In [9]:
Sector= {}
def save_nifty50():
    import pickle
    import csv
    tickers = []
    with open('ind_nifty500list.csv') as nifty50:
        csv_reader = csv.reader(nifty50, delimiter= ',')
        next(csv_reader)
        for i in csv_reader:
            tickers.append(i[2])
            Sector[i[2]] = i[1]
    tickers.append("NIFTY")
    with open("nifty500.pickle", "wb") as f:
        pickle.dump(tickers, f)
    return tickers
tickers  =  save_nifty50()
size_zero = 0
incorrect_data = 0 

def get_data(reload_nifty50 = False): 
    for ticker in tqdm(tickers):

        if os.path.exists('stocks_dfs/{}.csv'.format(ticker)):
            main_df = pd.read_csv('stocks_dfs/{}1.csv'.format(ticker), header= 0)    
            main_df.set_index('Date', inplace = True)
            if main_df.size == 0:
                print(ticker, 'zero')
                tickers.remove(ticker)
                continue            
            extracted_date = dt.datetime.strptime(main_df.index[-1] , '%Y-%m-%d').date() +  dt.timedelta(days=1)
            #print(ticker, extracted_date)
            if extracted_date <= dt.date.today():
                if ticker == 'NIFTY':
                    new_df = get_history(symbol=ticker, start = extracted_date, end = dt.date.today(), index = True)
                else:
                    new_df = get_history(symbol=ticker, start = extracted_date, end = dt.date.today())
                main_df = main_df.append(new_df)
                main_df = generate_data(main_df)
                if main_df.index[-1] != (end_date):
                    print(ticker, 'incomplete')
                    tickers.remove(ticker)
                    continue
                main_df.to_csv('stocks_dfs/{}.csv'.format(ticker))
                complete_results[ticker] = copy.deepcopy(company_template)    
                
get_data()


 15%|███████████▊                                                                     | 73/502 [00:58<04:36,  1.55it/s]

BBTC zero


 43%|██████████████████████████████████                                              | 214/502 [02:48<03:02,  1.58it/s]

IRB zero


 59%|███████████████████████████████████████████████▍                                | 298/502 [03:52<02:10,  1.57it/s]

...

NIITTECH incomplete


 69%|███████████████████████████████████████████████████████▎                        | 347/502 [04:27<01:49,  1.42it/s]

PVR zero


 99%|███████████████████████████████████████████████████████████████████████████████ | 496/502 [06:17<00:04,  1.31it/s]


FileNotFoundError: [Errno 2] File stocks_dfs/NIFTY1.csv does not exist: 'stocks_dfs/NIFTY1.csv'

In [29]:
for ticker in tqdm(tickers):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    df.index = df.Date
    if df.size == 0:
        print(ticker, 'zero')
        tickers.remove(ticker)
        continue
            
    if df.index[-1] != str(end_date):
        print(ticker, 'incomplete')
        tickers.remove(ticker)
        continue
    complete_results[ticker] = copy.deepcopy(company_template)

100%|███████████████████████████████████████████████████████████████████████████████| 493/493 [00:03<00:00, 141.43it/s]


In [30]:
print(len(tickers))

493


In [32]:
strat_inc_dec_results = {'increasing': [], 'decreasing': []}
def inc_dec(marange): #["Close", "3ma", "5ma" , "8ma"]
    increasing = {}
    decreasing = {}
    for ma in marange:
        if(ma == 'Close'):
            continue
        increasing[ma] = []
        decreasing[ma] = []   
    date_today = dt.date.today()
    for ticker in tickers:
        today_index, yesterday_index, yesterday2_index = get_index(ticker, ondate)
        if today_index == 0:
            continue
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        df.set_index('Date', inplace= True)        
        today = df.iloc[today_index]
        inc = 1
        dec = 1
        for i in range(0, len(marange)-1):
            if(today[marange[i]] >= today[marange[i+1]] and inc ==1):
                increasing[marange[i+1]].append(ticker)
            else:
                inc = 0
                break
                
        for i in range(0, len(marange)-1):
            if(today[marange[i]] < today[marange[i+1]] and dec == 1):
                decreasing[marange[i+1]].append(ticker)
            else:
                dec = 0
                break
    strat_inc_dec_results['increasing'] = increasing
    strat_inc_dec_results['decreasing'] = decreasing

def runstrat_incdec(marange, print_results = False):
    inc_dec(marange)
    if print_results:
        for i in strat_inc_dec_results['increasing']:
            print()
            print(i)
            print("increasing " , len(strat_inc_dec_results['increasing'][i]) ,"->>", strat_inc_dec_results['increasing'][i])
            print()
            print("decreasing " , len(strat_inc_dec_results['decreasing'][i]) ,"->>", strat_inc_dec_results['decreasing'][i])

#FIRST RESULT  
runstrat_incdec(["Close","3ma","5ma", "8ma", "14ma", "21ma", "50ma", "100ma" , "200ma"], print_results = True)
for ticker in strat_inc_dec_results['increasing']['200ma']:
    complete_results[ticker]['strat_incdec']['0-200'] = 5

for ticker in strat_inc_dec_results['increasing']['100ma']:
    complete_results[ticker]['strat_incdec']['0-100'] = 4
        
for ticker in strat_inc_dec_results['decreasing']['200ma']:
    complete_results[ticker]['strat_incdec']['0-200'] = -5

for ticker in strat_inc_dec_results['decreasing']['100ma']:
    complete_results[ticker]['strat_incdec']['0-100'] = -4

    
#Second Result        
runstrat_incdec(["Close","8ma", "14ma", "21ma"], print_results = False)
for ticker in strat_inc_dec_results['increasing']['21ma']:
    complete_results[ticker]['strat_incdec']['8-21'] = 3
        
for ticker in strat_inc_dec_results['decreasing']['21ma']:
    complete_results[ticker]['strat_incdec']['8-21'] = -3
    
#Trird Result
runstrat_incdec(["Close","50ma", "100ma", "200ma"], print_results = False)
for ticker in strat_inc_dec_results['increasing']['200ma']:
    complete_results[ticker]['strat_incdec']['50-200'] = 1
        
for ticker in strat_inc_dec_results['decreasing']['200ma']:
    complete_results[ticker]['strat_incdec']['50-200'] = -1


3ma
increasing  335 ->> ['3MINDIA', 'AIAENG', 'APLAPOLLO', 'AARTIIND', 'ABBOTINDIA', 'ADANIGREEN', 'ADANIPORTS', 'ADANIPOWER', 'ADANITRANS', 'ABFRL', 'ADVENZYMES', 'AJANTPHARM', 'AKZOINDIA', 'APLLTD', 'ALKEM', 'ALLCARGO', 'AMARAJABAT', 'AMBER', 'APOLLOHOSP', 'APOLLOTYRE', 'ARVINDFASN', 'ASIANPAINT', 'ASTRAZEN', 'ASTRAL', 'ATUL', 'AUROPHARMA', 'AVANTIFEED', 'AXISBANK', 'BASF', 'BEML', 'BSE', 'BAJAJ-AUTO', 'BAJFINANCE', 'BAJAJFINSV', 'BALKRISIND', 'BALMLAWRIE', 'BANKBARODA', 'BATAINDIA', 'BAYERCROP', 'BERGEPAINT', 'BDL', 'BEL', 'BHARATFORG', 'BPCL', 'BHARTIARTL', 'INFRATEL', 'BIOCON', 'BIRLACORPN', 'BSOFT', 'BLISSGVS', 'BLUESTARCO', 'BOMDYEING', 'BOSCHLTD', 'BRITANNIA', 'CCL', 'CRISIL', 'CANFINHOME', 'CANBK', 'CAPLIPOINT', 'CGCL', 'CARBORUNIV', 'CASTROLIND', 'CENTRALBK', 'CDSL', 'CENTURYPLY', 'CERA', 'CHAMBLFERT', 'CHOLAFIN', 'CIPLA', 'CUB', 'COCHINSHIP', 'COLPAL', 'COROMANDEL', 'CROMPTON', 'CYIENT', 'DLF', 'DABUR', 'DALBHARAT', 'DEEPAKNTR', 'DIVISLAB', 'DIXON', 'LALPATHLAB', 'DRREDDY',

In [33]:
strat_point5_results = {}
import math
def deviation(today, ndma, pcts):
    n = len(pcts)
    #[]
    diff = today[ndma] - today['Close']
    dev = ((diff/today['Close'])*100)
    if -pcts[n-1] < dev <= -pcts[n-2]:
        dev = -pcts[n-1]
    elif -pcts[n-2] < dev <= -pcts[n-3]:
        dev = -pcts[n-2]
    elif -pcts[n-3] < dev < -pcts[n-4]:
        dev = -pcts[n-3]
    elif -pcts[n-4] < dev < 0:
        dev = -pcts[n-4]
    elif 0 < dev <= pcts[n-4]:
        dev = pcts[n-4]
    elif pcts[n-4] < dev <= pcts[n-3]:
        dev = pcts[n-3]
    elif pcts[n-3] < dev < pcts[n-2]:
        dev = pcts[n-2]
    elif pcts[n-2] < dev < pcts[n-1]:
        dev = pcts[n-1]
    #print(dev)
    return dev

def mov_avg(pcts, mas, tickers):
    movavg_results = {}
    for pct in pcts:
        movavg_results[pct] = {}
        for ma in mas:
            movavg_results[pct][ma] = []
    
    for company in tickers:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(company), header= 0)
        today_index, yesterday_index, yesterday2_index = get_index(company, ondate)
        if today_index == 0:
            continue
        today = (df.iloc[today_index])
        i = 0
        for ma in mas:
            mad = deviation(today, ma, pcts)
            if abs(mad) in movavg_results:
                if(mad < 0):
                    company1 = "+"+company
                else:
                    company1 = "-"+ company
                movavg_results[abs(mad)][mas[i]].append(company1)
            i+=1 
    return movavg_results   

def runstrat_point5(pcts, mas,tickers, show_table = False):
    movavg_results = mov_avg(pcts, mas,tickers)
    if show_table == False:
        return
        
    companys = []
    print(movavg_results)
    print("Enter Weights of MAS : ")
    weights = []
    for i in range(0,len(mas)):
        #wt = int(input("Weight for mas[i] : "))
        wt = 1
        weights.append(wt)

    for i in pcts:
        for j in mas:
            for c in movavg_results[i][j]:
                companys.append(c[1:])

    companys = list(set(companys))    
    range_table = {'Company':[],'cscore':[], 'Overview':[]}
    for pct in pcts:
        name = 'RANGE : -{}-+{}'.format(pct, pct)
        range_table[name] = []
        
    key = list(range_table.keys())
    
    for company in companys:
        newcomp = []
        score = 0
        pos_score = 0
        neg_score = 0
        for i in range(0, len(pcts)):
            newstr = ""
            for j  in range(0, len(mas)):
                if '+'+company in movavg_results[pcts[i]][mas[j]]:
                    newstr = newstr+',     '+ '+' +mas[j]
                    pos_score =pos_score+ (len(pcts)-i)*weights[j]
                if '-'+company  in movavg_results[pcts[i]][mas[j]]:
                    newstr = newstr+',     '+ '-'+mas[j]
                    neg_score =neg_score+ (len(pcts)-i)*weights[j]
            newcomp.append(newstr[2:])
        
        overview = "+{} | -{}".format(pos_score, neg_score)
        newcomp.insert(0,overview)
        score = abs(pos_score+neg_score)
        newcomp.insert(0, score)
        newcomp.insert(0, company)
        for i in range(0, len(newcomp)):
            range_table[key[i]].append(newcomp[i])   
    table = pd.DataFrame.from_dict(range_table)
    table = table.set_index('Company')
    table = table.sort_values(by='cscore', ascending=False)
    table = table.drop(columns = ['cscore'])
    display(table)
    name = 'maranges_(10-7-20).csv'
    #table.to_csv(name)
    return table

In [34]:
pcts = [0.25,0.5]
mas = ["8ma", "14ma", "21ma", "50ma", "100ma", "200ma"]
movavg_table = runstrat_point5(pcts, mas, tickers, show_table = True)

{0.25: {'8ma': ['-BRITANNIA', '+LALPATHLAB', '+FDC', '+GAIL', '+GLAXO', '-GESHIP', '+IGL', '+JBCHEPHARM', '+LTI', '-MARUTI', '-WESTLIFE'], '14ma': ['-JAMNAAUTO', '-MPHASIS'], '21ma': ['-HINDUNILVR'], '50ma': [], '100ma': [], '200ma': ['-LALPATHLAB', '+THYROCARE', '-TORNTPHARM']}, 0.5: {'8ma': ['-AMBER', '+CERA', '-GMRINFRA', '+ICICIGI', '-MARICO', '+MUTHOOTFIN', '-TATASTLBSL'], '14ma': [], '21ma': [], '50ma': ['-OMAXE'], '100ma': ['+ALKEM'], '200ma': ['+PFIZER']}}
Enter Weights of MAS : 


,Overview,RANGE : -0.25-+0.25,RANGE : -0.5-+0.5
Company,,,
LALPATHLAB,+2 | -2,"+8ma, -200ma",
MARUTI,+0 | -2,-8ma,
TORNTPHARM,+0 | -2,-200ma,
WESTLIFE,+0 | -2,-8ma,
IGL,+2 | -0,+8ma,
FDC,+2 | -0,+8ma,
MPHASIS,+0 | -2,-14ma,
JAMNAAUTO,+0 | -2,-14ma,
THYROCARE,+2 | -0,+200ma,


In [35]:
strat_point5all_results = {'near_close':[], 'above_close': [], 'below_close':[]}

def strat_point5all_close(pcts, mas, tickers, show_table = False):
    movavg_results = mov_avg(pcts, mas, tickers)
    companys = []
    print("Enter Weights of MAS : ")
    weights = []
    for i in range(0,len(mas)):
        #wt = int(input('Weight for {}:'.format(mas[i])))
        wt = 1
        weights.append(wt)

    for i in pcts:
        for j in mas:
            for c in movavg_results[i][j]:
                companys.append(c[1:])

    companys = list(set(companys))    
    range_table = {'Company':[],'cscore':[], 'Overview':[]}
    for pct in pcts:
        name = 'RANGE : -{}-+{}'.format(pct, pct)
        range_table[name] = []
        
    key = list(range_table.keys())
    
    for company in companys:
        newcomp = []
        occourence = []
        score = 0
        pos_score = 0
        neg_score = 0
        for i in range(0, len(pcts)):
            newstr = ""
            for j  in range(0, len(mas)):
                if '+'+company in movavg_results[pcts[i]][mas[j]]:
                    newstr = newstr+',     '+ '+' +mas[j]
                    occourence.append(mas[j])
                    pos_score =pos_score+ (len(pcts)-i)*weights[j]
                if '-'+company  in movavg_results[pcts[i]][mas[j]]:
                    newstr = newstr+',     '+ '-'+mas[j]
                    occourence.append(mas[j])
                    neg_score =neg_score+ (len(pcts)-i)*weights[j]
            newcomp.append(newstr[2:])
        
        overview = "+{} | -{}".format(pos_score, neg_score)
        newcomp.insert(0,overview)
        score = abs(pos_score+neg_score)
        newcomp.insert(0, score)
        newcomp.insert(0, company)
        if(len(occourence) < len(mas)):
            continue
        for i in range(0, len(newcomp)):
            range_table[key[i]].append(newcomp[i])   
    table = pd.DataFrame.from_dict(range_table)
    table = table.set_index('Company')
    table = table.sort_values(by='cscore', ascending=False)
    table = table.drop(columns = ['cscore'])
    if show_table :
        display(table)
    #table.to_csv('maranges_81421.csv')
    return table

pcts = [0.25, 0.5]
mas = ['8ma', '14ma', '21ma']
point5all_table = strat_point5all_close(pcts, mas, tickers, show_table = False)
point5all_companys = list(point5all_table.index)
for company in point5all_companys:
    strat_point5all_results['near_close'].append(company)

def strat_point5all_anywhere(pct, mas, show_table = False):
    companys = []
    for company in tickers:
        today_index, yesterday_index, yesterday2_index = get_index(company, ondate)
        if today_index == 0:
            continue
        df = pd.read_csv('stocks_dfs/{}.csv'.format(company), header= 0)
        today = (df.iloc[today_index])
        values = []
        for ma in mas:
            values.append(today[ma])
        max_val = max(values)
        min_val = min(values)
        mid = (max_val + min_val)/2
        if( (max_val - mid)/mid < (pct/200)  and (mid - min_val)/mid < (pct/200) ):
            #print(company ,((max_val - mid)/mid)*100, ((mid - min_val)/mid)*100)
            if(min_val > today.Close):
                strat_point5all_results['above_close'].append(company)
            if(max_val < today.Close):
                strat_point5all_results['below_close'].append(company)                
            companys.append(company)
    if(show_table):
        display(companys)
    return companys

mas = ["8ma", "14ma" , "21ma"]
companys = strat_point5all_anywhere(0.5, mas, show_table = False)
print("8/14/21 MAS near each other anywhere irrespective of trade value")
for company in strat_point5all_results['near_close']:
    complete_results[company]['BREAKPOINTS'] += "Point5 near close "

for company in strat_point5all_results['above_close']:
    complete_results[company]['strat_point5all']['above_close'] = -2    

for company in strat_point5all_results['below_close']:
    complete_results[company]['strat_point5all']['below_close'] = 2        
    
print(strat_point5all_results)

Enter Weights of MAS : 
8/14/21 MAS near each other anywhere irrespective of trade value
{'near_close': [], 'above_close': [], 'below_close': []}


In [36]:
strat_sma_ema_cross_results = {'Bullish' : [], 'Bearish': []}
def runstrat_sma_ema_cross():
    cross = []
    for ticker in tickers:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        today_index, yesterday_index, yesterday2_index = get_index(ticker, ondate)
        if today_index == 0:
            continue
        df['ema10'] = df['Close'].ewm(span=10).mean()
        df['10ma'] = df['Close'].rolling(window = 10, min_periods = 0).mean()
        today = df.iloc[today_index]
        todaysma = (df.iloc[today_index]['10ma'])
        todayema = (df.iloc[today_index]['ema10'])
        yesterdaysma = (df.iloc[yesterday_index]['10ma'])
        yesterdayema = (df.iloc[yesterday_index]['ema10'])
        tdiff = todayema - todaysma
        ydiff = yesterdayema - yesterdaysma
        if(ydiff*tdiff < 0):
            if tdiff > 0:
                cross.append("+"+ticker)
                if(todaysma > today.Close):
                    complete_results[ticker]['strat_smaema'] = 2
                elif 1.1*today.Close> todaysma > 0.9*today.Close:
                    complete_results[ticker]['strat_smaema'] = 2
                else:
                    complete_results[ticker]['strat_smaema'] = 2
                strat_sma_ema_cross_results['Bullish'].append(ticker)
            else:
                cross.append("-"+ticker)
                if(todaysma < today.Close):
                    complete_results[ticker]['strat_smaema'] = -2
                elif 1.1*today.Close> todaysma > 0.9*today.Close:
                    complete_results[ticker]['strat_smaema'] = -2
                else:
                    complete_results[ticker]['strat_smaema'] = -2
                strat_sma_ema_cross_results['Bearish'].append(ticker)
    return cross
        
cross = runstrat_sma_ema_cross()
print("SMA|EMA CROSSOVERS")
print(cross)   

SMA|EMA CROSSOVERS
['+3MINDIA', '+APLAPOLLO', '+AARTIIND', '+ABBOTINDIA', '+ADANIPORTS', '+ABCAPITAL', '+ADVENZYMES', '+APLLTD', '+ALKEM', '+AMBER', '+ASHOKA', '+ASTERDM', '+ATUL', '+AUROPHARMA', '+BSE', '+BAJAJ-AUTO', '+BALKRISIND', '+BALRAMCHIN', '+MAHABANK', '+BHARTIARTL', '+BSOFT', '+CCL', '+CANFINHOME', '+CGCL', '+CDSL', '+CENTURYPLY', '+CHALET', '+DABUR', '+DALBHARAT', '+DELTACORP', '+EIDPARRY', '+EIHOTEL', '+EMAMILTD', '+ERIS', '+ESSELPACK', '+FDC', '+GILLETTE', '+GLAXO', '+GODREJAGRO', '+GODREJCP', '+GODREJIND', '+GODREJPROP', '+GREAVESCOT', '+GSFC', '+GULFOILLUB', '+HDFCAMC', '+HDFCLIFE', '+HEIDELBERG', '+HEROMOTOCO', '+HEXAWARE', '+HINDALCO', '+HINDCOPPER', '+HDFC', '+ICICIPRULI', '+ICRA', '+IDFCFIRSTB', '+IDFC', '+IIFL', '+IBREALEST', '+IOB', '+INFIBEAM', '+JMFINANCIL', '+JSWSTEEL', '+JUBILANT', '+JUSTDIAL', '+JYOTHYLAB', '+KTKBANK', '+KOTAKBANK', '+L&TFH', '+LTTS', '+LICHSGFIN', '+LUXIND', '+MANAPPURAM', '+MFSL', '+MINDACORP', '+NIITTECH', '+NTPC', '+NBVENTURES', '+NESTLEIN

In [37]:
strat_macd_results = {'bullish_crossover' : [], 'bearish_crossover': [], 'middle_crossover_pos': [], 'middle_crossover_neg': [], 'hist_bearish_turn': [], 'hist_bullish_turn': []}
def macd_study():
    for ticker in tickers:        
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        today_index, yesterday_index, yesterday2_index = get_index(ticker, ondate)
        if today_index == 0:
            continue
        exp1 = df['Close'].ewm(span=12, adjust=False).mean()
        exp2 = df['Close'].ewm(span=26, adjust=False).mean()
        macd = exp1-exp2
        signal = macd.ewm(span=9, adjust=False).mean()
        df['hist'] = macd-signal
        hist = TA.SMA(df, 3, "hist")
        today_hist = hist[today_index]
        yesterday_hist1 = hist[yesterday_index]
        yesterday_hist2 = hist[yesterday2_index]
        today_macd = macd[today_index]
        yesterday_macd = macd[yesterday_index]
        today_signal = signal[today_index]
        yesterday_signal = signal[yesterday_index]
        tdiff = today_macd - today_signal
        ydiff = yesterday_macd - yesterday_signal
        if(today_hist > 0 and yesterday_hist2 < yesterday_hist1 and yesterday_hist1 > today_hist):
            strat_macd_results['hist_bearish_turn'].append(ticker)
        if(today_hist < 0 and yesterday_hist2 > yesterday_hist1 and yesterday_hist1 < today_hist):
            strat_macd_results['hist_bullish_turn'].append(ticker)
        if tdiff*ydiff < 0:
            if tdiff > 0 :
                strat_macd_results['bullish_crossover'].append(ticker)
            if tdiff < 0 :
                strat_macd_results['bearish_crossover'].append(ticker)
        if(today_macd < 0 and yesterday_macd > 0):
            strat_macd_results['middle_crossover_neg'].append(ticker)
        elif(today_macd > 0 and yesterday_macd < 0):
            strat_macd_results['middle_crossover_pos'].append(ticker)                
        
            
print("MACD NEAR 0")  

macd_study()
for company in strat_macd_results['bullish_crossover']:
    complete_results[company]['strat_macd']['normal_crossover'] =2
for company in strat_macd_results['bearish_crossover']:
    complete_results[company]['strat_macd']['normal_crossover'] = -2
for company in strat_macd_results['middle_crossover_neg']:
    complete_results[company]['strat_macd']['middle_crossover'] = -3
for company in strat_macd_results['middle_crossover_pos']:
    complete_results[company]['strat_macd']['middle_crossover'] = 3
for company in strat_macd_results['hist_bullish_turn']:
    complete_results[company]['strat_macd']['hist_turnover'] = 1
for company in strat_macd_results['hist_bearish_turn']:
    complete_results[company]['strat_macd']['hist_turnover'] = -1
print(strat_macd_results)

MACD NEAR 0
{'bullish_crossover': ['BASF', 'DISHTV', 'ENGINERSIN', 'JSWENERGY', 'J&KBANK', 'OIL', 'PARAGMILK', 'TRIDENT'], 'bearish_crossover': ['OMAXE'], 'middle_crossover_pos': [], 'middle_crossover_neg': ['IPCALAB'], 'hist_bearish_turn': ['BASF', 'NATIONALUM'], 'hist_bullish_turn': ['3MINDIA', 'AIAENG', 'AARTIIND', 'ADANIPORTS', 'AMARAJABAT', 'BAJFINANCE', 'BOSCHLTD', 'BRITANNIA', 'CRISIL', 'CHAMBLFERT', 'LALPATHLAB', 'EDELWEISS', 'ERIS', 'FDC', 'FSL', 'GET&D', 'GALAXYSURF', 'GICRE', 'GILLETTE', 'GODFRYPHLP', 'GODREJIND', 'GODREJPROP', 'GRANULES', 'GESHIP', 'HDFCBANK', 'HDFCLIFE', 'ICICIBANK', 'ICICIPRULI', 'INFIBEAM', 'NAUKRI', 'JKCEMENT', 'KEI', 'KALPATPOWR', 'LICHSGFIN', 'MARUTI', 'MINDAIND', 'MUTHOOTFIN', 'NESCO', 'NH', 'ORIENTELEC', 'PERSISTENT', 'QUESS', 'RAJESHEXPO', 'SHOPERSTOP', 'SHREECEM', 'RENUKA', 'SRTRANSFIN', 'SUNTV', 'SUNCLAYLTD', 'SUPREMEIND', 'SYNGENE', 'TATAELXSI', 'TITAN', 'TRENT', 'UJJIVAN', 'VRLLOG', 'WABCOINDIA', 'ZYDUSWELL']}


In [38]:
def run_strat_RSI(series, period=14, show_results = False):
    delta = series.diff().dropna()
    ups = delta * 0
    downs = ups.copy()
    ups[delta > 0] = delta[delta > 0]
    downs[delta < 0] = -delta[delta < 0]
    ups[ups.index[period-1]] = np.mean( ups[:period] ) #first value is sum of avg gains
    ups = ups.drop(ups.index[:(period-1)])
    downs[downs.index[period-1]] = np.mean( downs[:period] ) #first value is sum of avg losses
    downs = downs.drop(downs.index[:(period-1)])
    rs = ups.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() / \
         downs.ewm(com=period-1,min_periods=0,adjust=False,ignore_na=False).mean() 
    x = 100 - 100 / (1 + rs)
    return x 

for ticker in tickers:
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)        
    x = run_strat_RSI(df.Close, 14)
    break;

In [39]:
strat_stochrsi_results = {'negcrossover':  [], 'poscrossover' : [], 'negmiddle' : [], 'posmiddle' : []}
stochrsi_K = []
def run_strat_stochrsi(ticker):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)        
    today_index, yesterday_index, yesterday2_index = get_index(ticker, ondate)
    if today_index == 0 :
        return
    df = df.rename(columns = {"Close":"close", "Open" : "open", "High" : "high", "Low": "low", "Volume": "volume"}) 
    rsi = run_strat_RSI(df.close, 14)
    stochrsi  = (rsi - rsi.rolling(14).min()) / (rsi.rolling(14).max() - rsi.rolling(14).min())
    stochrsi_K = stochrsi.rolling(3).mean()
    stochrsi_D = stochrsi_K.rolling(3).mean()
    todayK = stochrsi_K[today_index]
    todayD = stochrsi_D[today_index]
    yesterdayK = stochrsi_K[yesterday_index]
    yesterdayD = stochrsi_D[yesterday_index]
    tdiff = todayK - todayD
    ydiff = yesterdayK - yesterdayD
    
    if abs(todayK - todayD)/min(todayK, todayD) < 0.1 and 0.16<todayK<0.24 :
        strat_stochrsi_results['poscrossover'].append(ticker)
        complete_results[ticker]['strat_stochrsi']['Bullish'] = 3
    elif abs(todayK - todayD)/min(todayK, todayD) < 0.1 and 0.76<todayK<0.84 :
        strat_stochrsi_results['negcrossover'].append(ticker)
        complete_results[ticker]['strat_stochrsi']['Bearish'] = -3
    elif 0.16<todayK<0.2 and yesterdayK<todayK:
        strat_stochrsi_results['poscrossover'].append(ticker)
        complete_results[ticker]['strat_stochrsi']['Bullish'] = 2
    elif 0.8<todayK<0.84 and yesterdayK>todayK:
        strat_stochrsi_results['negcrossover'].append(ticker)
        complete_results[ticker]['strat_stochrsi']['Bearish'] = -2
    elif 0.2<todayK<0.24 and yesterdayK>todayK:
        complete_results[ticker]['OverBuySell'] += 'Getting OverSold  '
    elif 0.8<todayK<0.84 and yesterdayK<todayK:
        complete_results[ticker]['OverBuySell'] += 'Getting OverBought  '
        
        
        
    if(ydiff*tdiff < 0):
        if tdiff < 0 and yesterdayK > 0.8:
            strat_stochrsi_results['negcrossover'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Bearish'] = -3
        elif tdiff > 0 and yesterdayK > 0.8:
            strat_stochrsi_results['negcrossover'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Bullish'] = 1
        elif tdiff < 0 and yesterdayK < 0.8:
            strat_stochrsi_results['negmiddle'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Middle_cross'] = -1
        elif tdiff > 0 and yesterdayK < 0.2:
            strat_stochrsi_results['poscrossover'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Bullish'] = 3
        elif tdiff < 0 and yesterdayK < 0.2:
            strat_stochrsi_results['poscrossover'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Bullish'] = -1
        elif tdiff > 0 and yesterdayK > 0.2:
            strat_stochrsi_results['posmiddle'].append(ticker)
            complete_results[ticker]['strat_stochrsi']['Middle_cross'] = 1
    
for ticker in tickers:  
    run_strat_stochrsi(ticker)
print(strat_stochrsi_results)

<ipython-input-39-a8a1b9035ccf>:20: RuntimeWarning: divide by zero encountered in double_scalars
  if abs(todayK - todayD)/min(todayK, todayD) < 0.1 and 0.16<todayK<0.24 :
<ipython-input-39-a8a1b9035ccf>:23: RuntimeWarning: divide by zero encountered in double_scalars
  elif abs(todayK - todayD)/min(todayK, todayD) < 0.1 and 0.76<todayK<0.84 :


KeyError: 'OverBuySell'

In [ ]:
strat_hieken_ashi_results = {"SuperBullish" : [], "SuperBearish" : [], "Bullish" : [] , "Bearish": [], "Breakpoint" : []}

def heiken_ashi(ticker):    
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)    
    df['HA_Close']=(df['Open']+ df['High']+ df['Low']+df['Close'])/4
    idx = df.index.name
    df.reset_index(inplace=True)
    for i in range(0, len(df)):
        if i == 0:
            df.at[i, 'HA_Open'] = ((df.at[i, 'Open'] + df.at[i, 'Close']) / 2)
        else:
            df.at[i, 'HA_Open'] =  ((df.at[i - 1, 'HA_Open'] + df.at[i - 1, 'HA_Close']) / 2)
    if idx:
        df.set_index(idx, inplace=True)
    df['HA_High']=df[['HA_Open','HA_Close','High']].max(axis=1)
    df['HA_Low']=df[['HA_Open','HA_Close','Low']].min(axis=1)
    today_index, yesterday_index, yesterday2_index = get_index(ticker, ondate)
    if today_index == 0 :
        return
    df = df.iloc[today_index]
    if(df['HA_Low'] == df['HA_Open'] and df['HA_Close'] >= df['HA_Open'] + df['HA_Open']*0.04):
        strat_hieken_ashi_results["SuperBullish"].append(ticker)
    if(df['HA_Low'] == df['HA_Open'] and df['HA_Open'] + df['HA_Open']*0.02 <= df['HA_Close'] <= df['HA_Open'] + df['HA_Open']*0.04):
        strat_hieken_ashi_results["Bullish"].append(ticker)
    if(df['HA_High'] == df['HA_Open'] and df['HA_Close'] <= df['HA_Open'] - df['HA_Open']*0.04):
        strat_hieken_ashi_results["SuperBearish"].append(ticker)
    if(df['HA_High'] == df['HA_Open'] and df['HA_Open'] - df['HA_Open']*0.04 < df['HA_Close'] <= df['HA_Open'] - df['HA_Open']*0.02):
        strat_hieken_ashi_results["Bearish"].append(ticker)
    body = abs(df['HA_Open'] - df['HA_Close'])
    if(df['HA_High'] >= df['HA_Open'] + body and df['HA_Low'] <= df['HA_Open'] - body):
        strat_hieken_ashi_results["Breakpoint"].append(ticker)    
        

for company in tickers:
    heiken_ashi(company)

for x in strat_hieken_ashi_results:
    for ticker in strat_hieken_ashi_results[x]:
        if x == 'SuperBullish':
            complete_results[ticker]['strat_hieken_ashi']['Bullish'] = 3
        elif x == 'Bullish':
            complete_results[ticker]['strat_hieken_ashi']['Bullish'] = 2
        elif x == 'SuperBearish':
            complete_results[ticker]['strat_hieken_ashi']['Bearish'] = -3
        elif x == 'Bearish':
            complete_results[ticker]['strat_hieken_ashi']['Bearish'] = -2
        elif x == 'Breakpoint':
            complete_results[ticker]['BREAKPOINTS'] += "HA Breakpoint"
    

In [ ]:
display(strat_hieken_ashi_results)

In [ ]:
strat_fisher_results = {'negcrossover': [], 'poscrossover': [],'neartopline':[] , 'nearbotline': [], 'negmiddle':[], 'posmiddle': [] }
def fish(tickers):
    negcrossover = []
    poscrossover = []
    neartopline = []
    nearbotline = []
    negmiddle = []
    posmiddle = []
    for ticker in tickers:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        today_index, yesterday_index, yesterday2_index = get_index(ticker, ondate)
        if today_index == 0:
            continue
        df = df.rename(columns = {"Close":"close", "Open" : "open", "High" : "high", "Low": "low", "Volume": "volume"}) 
        #idx = df.index.name
        #df.reset_index(inplace=True)
        df['fish'] = TA.FISH(df, 10)
        #df['fish'] = TA.SMA(df, 3, "fish")    
        today = df.iloc[today_index]['fish']
        yesterday = df.iloc[yesterday_index]['fish']
        yesterday2 = df.iloc[yesterday2_index]['fish']
        if(yesterday2 < yesterday and yesterday >= today):
            if(today > 1.45):
                negcrossover.append(ticker)
            elif  -1.45 < today < 1.45:
                negmiddle.append(ticker)
        if(yesterday2 > yesterday and yesterday <= today):
            if(today < -1.45):
                poscrossover.append(ticker)
            elif  -1.45 < today < 1.45:
                posmiddle.append(ticker)

        if(yesterday2 > yesterday and  1.55 > today > 1.45 ):
            neartopline.append(ticker)
        if(yesterday2 < yesterday and  -1.55 < today < -1.45 ):
            nearbotline.append(ticker) 
    strat_fisher_results['negcrossover']= negcrossover
    strat_fisher_results['poscrossover']= poscrossover
    strat_fisher_results['neartopline']= neartopline
    strat_fisher_results['nearbotline']= nearbotline
    strat_fisher_results['negmiddle']= negmiddle
    strat_fisher_results['posmiddle']= posmiddle
    
fish(tickers)
for x in strat_fisher_results:
    for ticker in strat_fisher_results[x]:
        if x == 'poscrossover':
            complete_results[ticker]['strat_fisher']['Bullish'] = 2
        elif x == 'negcrossover':
            complete_results[ticker]['strat_fisher']['Bearish'] = -2
        elif x == 'neartopline':
            complete_results[ticker]['strat_fisher']['Bearish'] = -3
        elif x == 'nearbotline':
            complete_results[ticker]['strat_fisher']['Bullish'] = 3
        elif x == 'negmiddle':
            complete_results[ticker]['strat_fisher']['Middle_cross'] = -1
        elif x == 'posmiddle':
            complete_results[ticker]['strat_fisher']['Middle_cross'] = 1
            
fishpos = set(strat_fisher_results['poscrossover'])|set(strat_fisher_results['nearbotline'])|set(strat_fisher_results['posmiddle'])
fishneg = set(strat_fisher_results['negcrossover'])|set(strat_fisher_results['neartopline'])|set(strat_fisher_results['negmiddle'])

In [ ]:
display(strat_fisher_results)

In [ ]:
strat_delivery_results = {}
def strat_delivery():
    for ticker in tickers[:-1]:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
        today_index, yesterday_index, yesterday2_index = get_index(ticker, ondate)
        if today_index ==0:
            continue
        deliverable_5 = df['%Deliverble'].rolling(window=5, min_periods=0).mean().iloc[yesterday_index]
        Volume_5 = df['Volume'].rolling(window=5, min_periods=0).mean().iloc[yesterday_index]
        today = df.iloc[today_index]
        if today['Volume'] > Volume_5*2 or today['%Deliverble'] > deliverable_5*1.5:
            strat_delivery_results[ticker]= {'pricechange': 0, 'Remarks' : ""}
            if today['Volume'] > Volume_5*3:
                strat_delivery_results[ticker]['Remarks'] += "3 times Volume  "
            elif today['Volume'] > Volume_5*2:
                strat_delivery_results[ticker]['Remarks'] += "2 times Volume  "
            if today['%Deliverble'] > deliverable_5*2:
                strat_delivery_results[ticker]['Remarks'] += "2 times Delivery"
            elif today['%Deliverble'] > deliverable_5*1.5:
                strat_delivery_results[ticker]['Remarks'] += "1.5 times Delivery"
            change_cost = (df.iloc[today_index]['Close'] - df.iloc[yesterday_index]['Close'])/df.iloc[yesterday_index]['Close']
            strat_delivery_results[ticker]['pricechange'] = round(change_cost*100,2)
            complete_results[ticker]['REMARKS'] = strat_delivery_results[ticker]['Remarks']
            print(ticker, change_cost*100)
        
strat_delivery()
display(strat_delivery_results)

In [ ]:
mov_avg_table = strat_point5all_close([0.25, 0.5], ["8ma", "14ma", "21ma"], tickers) #mov_avg_81421_near_close
companys_inpoint5_range_anywhere = strat_point5all_anywhere(0.5, ["8ma", "14ma", "21ma"]) #mov_avg_81421_anywhere
A_intersection_B = (set(mov_avg_table.index) & (set(companys_inpoint5_range_anywhere))) #A_intersection_B
A_union_B = (set(mov_avg_table.index) | (set(companys_inpoint5_range_anywhere))) #A_union_B

#SMA/EMA10 CROSSOVER
sma_ema = runstrat_sma_ema_cross()
c1 = []
c2 = []
for company in sma_ema:
    if company[0] == '+':
        c1.append(company[1:]) #(+ve crossover sma10/ema10)
    else:
        c2.append(company[1:]) #(-ve crossover sma10/ema10)

#increasing/decreasing        
inc_dec(["Close","3ma", "5ma", "8ma", "14ma", "21ma", "50ma", "100ma", "200ma"])        
x1 = strat_inc_dec_results['increasing']["21ma"]
y1 = strat_inc_dec_results['decreasing']["21ma"]
x2 = strat_inc_dec_results['increasing']["50ma"]
y2 = strat_inc_dec_results['decreasing']["50ma"]
x3 = strat_inc_dec_results['increasing']["100ma"]
y3 = strat_inc_dec_results['decreasing']["100ma"]
x4 = strat_inc_dec_results['increasing']["200ma"]
y4= strat_inc_dec_results['decreasing']["200ma"]
inc_dec(["Close", "8ma", "14ma", "21ma"])
x5 = strat_inc_dec_results['increasing']["21ma"]
y5 = strat_inc_dec_results['decreasing']["21ma"]
inc_dec(["Close", "50ma", "100ma", "200ma"])
x6 = strat_inc_dec_results['increasing']["200ma"]
y6 = strat_inc_dec_results['decreasing']["200ma"]

#intersecion/union of incdec and SMA/EMA crossover

In [ ]:
p16 = set(c1)&set(x6)
q16 = set(c2)&set(y6)
p13 = set(c1)&set(x3)
q13 = set(c2)&set(y3)
p15 = set(c1)&set(x5)
q15 = set(c2)&set(y5)
p13 = set(c1)&set(x3)
q13 = set(c2)&set(y3) 

super_bullish_smaema_incdec = ((set(x6)|set(x5)|set(x3)) & set(c1))|set(x4)
super_bearish_smaema_incdec = ((set(y6)|set(y5)|set(y3)) & set(c2))|set(y4)
very_bullish_smaema_incdec = ((set(x6)|set(x5)) & set(c1))|set(x3)
very_bearish_smaema_incdec = ((set(y6)|set(y5)) & set(c2))|set(y3)
bullish_smaema_incdec = ((set(x6)&set(x5)) | set(c1))|set(x3)
bearish_smaema_incdec= ((set(y6)&set(y5)) | set(c2))|set(y3)
fish_ints_incdec_pos = (set(x3)|set(x5)|set(x6))& fishpos
fish_ints_incdec_neg = (set(x3)|set(y5)|set(y6))& fishneg
A_ints_B_ints_fish = A_union_B & (fishpos | fishneg)

heikenashi_smaema_incdec_superbullish_pos = set(strat_hieken_ashi_results['SuperBullish']) & set(super_bullish_smaema_incdec)
heikenashi_smaema_incdec_verybullish_pos = set(strat_hieken_ashi_results['Bullish']) & set(very_bullish_smaema_incdec)
heikenashi_smaema_incdec_bullish_pos = set(strat_hieken_ashi_results['Bullish']) & set(bullish_smaema_incdec)
heikenashi_smaema_incdec_superbearish_pos = set(strat_hieken_ashi_results['SuperBearish']) & set(super_bearish_smaema_incdec)
heikenashi_smaema_incdec_verybearish_pos = set(strat_hieken_ashi_results['Bearish']) & set(very_bearish_smaema_incdec)
heikenashi_smaema_incdec_bearish_pos = set(strat_hieken_ashi_results['Bearish']) & set(bearish_smaema_incdec)
heikenashi_breakpoints_smaema_icndec_allbullish = set(strat_hieken_ashi_results['Breakpoint']) & (set(super_bullish_smaema_incdec)|set(very_bullish_smaema_incdec)|set(bullish_smaema_incdec))
heikenashi_breakpoints_smaema_icndec_allbearish = set(strat_hieken_ashi_results['Breakpoint']) & (set(super_bearish_smaema_incdec)|set(very_bearish_smaema_incdec)|set(bearish_smaema_incdec))
fish_heikenashi_pos = set(fishpos)&set(strat_hieken_ashi_results['Bullish'])
fish_heikenashi_neg = set(fishneg)&set(strat_hieken_ashi_results['Bearish'])

In [ ]:
print( "\nA(mov_avg_81421_near_close) ==>> ",list(mov_avg_table.index))
print("\nB(mov_avg_81421_anywhere) ==>> ", companys_inpoint5_range_anywhere)
print("\nA_intersection_B ==> ", A_intersection_B)
print("\nA_union_B ==>> ", A_union_B)
print("\nc1(+ve crossover sma10/ema10) ==>> " , c1 )
print("\nc2(-ve crossover sma10/ema10) ==>> " , c2 )
print("\nSMA/EMA intersection inc/dec super_bullish ==>> (increasing) ", len(super_bullish_smaema_incdec),  super_bullish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec super_bearish==>> (decreasing) ", len(super_bearish_smaema_incdec),  super_bearish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec very_bullish==>> (increasing) ", len(very_bullish_smaema_incdec),  very_bullish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec verybullish==>> (decreasing) ", len(very_bearish_smaema_incdec),  very_bearish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec bullish ==>> (increasing) ", len(bullish_smaema_incdec),  bullish_smaema_incdec )
print("\nSMA/EMA intersection inc/dec bullish ==>> (decreasing) ", len(bearish_smaema_incdec),  bearish_smaema_incdec )
print("\n Fisher incdec results positive ==>> ", len(fish_ints_incdec_pos), fish_ints_incdec_pos)
print("\n Fisher incdec results negative ==>> ", len(fish_ints_incdec_neg), fish_ints_incdec_neg)
print("\n Fisher ints 0.5 results ==>> ", len(A_ints_B_ints_fish), A_ints_B_ints_fish)

print("\n heikenashi_smaema_incdec_superbullish_pos ==>> ", len(heikenashi_smaema_incdec_superbullish_pos) , heikenashi_smaema_incdec_superbullish_pos)      
print("\n heikenashi_smaema_incdec_superbearish_pos ==>> ", len(heikenashi_smaema_incdec_superbearish_pos) , heikenashi_smaema_incdec_superbearish_pos)      

print("\n heikenashi_smaema_incdec_verybullish_pos ==>> ", len(heikenashi_smaema_incdec_verybullish_pos) , heikenashi_smaema_incdec_verybullish_pos)      
print("\n heikenashi_smaema_incdec_verybearish_pos ==>> ", len(heikenashi_smaema_incdec_verybearish_pos) , heikenashi_smaema_incdec_verybearish_pos)      

print("\n heikenashi_smaema_incdec_bullish_pos ==>> ", len(heikenashi_smaema_incdec_bullish_pos) , heikenashi_smaema_incdec_bullish_pos)      
print("\n heikenashi_smaema_incdec_bearish_pos ==>> ", len(heikenashi_smaema_incdec_bearish_pos) , heikenashi_smaema_incdec_bearish_pos)      

print("\n heikenashi_breakpoints_smaema_icndec_allbullish ==>> ", len(heikenashi_breakpoints_smaema_icndec_allbullish) , heikenashi_breakpoints_smaema_icndec_allbullish)      
print("\n heikenashi_breakpoints_smaema_icndec_allbearish ==>> ", len(heikenashi_breakpoints_smaema_icndec_allbearish) , heikenashi_breakpoints_smaema_icndec_allbearish)      

print("\nfish_heikenashi_pos ==>> ",len(fish_heikenashi_pos), fish_heikenashi_pos)
print("\nfish_heikenashi_neg ==>> ",len(fish_heikenashi_neg), fish_heikenashi_neg)



In [ ]:
table_results = {}
table_results['CScore'] = []
table_results['Sector'] = []
table_results['Score'] = []
table_results['Close'] = []
table_results['PctChange'] = []
table_results['Pos_Score'] = []
table_results['Neg_Score'] = []
table_results['Pos_Strat'] = []
table_results['Neg_Strat'] = []
table_results['Company'] = []
for strat in complete_results['HDFCBANK']:
    if strat == 'strat_smaema' or strat == 'REMARKS'  or strat == 'BREAKPOINTS' or strat == 'OverBuySell':
        table_results[strat] = []
        continue
    for category in complete_results['HDFCBANK'][strat]:
        name = strat+'('+category+')'
        table_results[name] = []

for company in complete_results:
    if company not in tickers:
        continue        
    if company in complete_results:
        table_results['Company'].append(company)
    if company == 'NIFTY':
        table_results['Sector'].append('-')
    else:
        table_results['Sector'].append(Sector[company])
    score = 0  
    cscore = 0
    pos_score = 0
    neg_score = 0
    pos_strat = 0
    neg_strat = 0
    for strat in complete_results[company]:
        df = pd.read_csv('stocks_dfs/{}.csv'.format(company), header= 0)

        if strat == 'REMARKS' or strat == 'BREAKPOINTS' or strat == 'OverBuySell':
            table_results[strat].append(complete_results[company][strat])
            continue
        if strat == 'strat_smaema':
            table_results[strat].append(complete_results[company][strat])
            score += complete_results[company][strat]
            cscore += abs(complete_results[company][strat])
            if complete_results[company][strat] < 0:
                neg_score += complete_results[company][strat]
                neg_strat += 1
            elif  complete_results[company][strat] >0:
                pos_score += complete_results[company][strat] 
                pos_strat += 1
            continue
        #print(company, strat , complete_results[company][strat])
        for category in complete_results[company][strat]:
            name = strat+'('+category+')'
            #print(company, strat, name, table_results[name])
            table_results[name].append(complete_results[company][strat][category])   
            score += complete_results[company][strat][category]
            cscore += abs(complete_results[company][strat][category])
            if complete_results[company][strat][category] < 0:
                neg_score += complete_results[company][strat][category]
                neg_strat += 1
            elif  complete_results[company][strat][category] >0:
                pos_score += complete_results[company][strat][category]
                pos_strat += 1
                
    table_results['Close'].append(df.iloc[-1].Close)
    table_results['PctChange'].append(round(((df.iloc[-1].Close - df.iloc[-2].Close)/df.iloc[-2].Close)*100, 2)) 
    table_results['Pos_Score'].append(pos_score)
    table_results['Neg_Score'].append(neg_score)
    table_results['Pos_Strat'].append(pos_strat)    
    table_results['Neg_Strat'].append(neg_strat)    
    table_results['CScore'].append('+' + str(pos_score) + ' | -' + str(neg_score))
    table_results['Score'].append(cscore)

table = pd.DataFrame.from_dict(table_results)
table = table.set_index('Company')
table = table.sort_values(by='Score', ascending=False)
table['Score'] = table['Pos_Score'] + table['Neg_Score']
table = table.rename(columns = {"CScore":" Net Score: High :  Bullish Low: Bearish midlle : Hold ",
                                'strat_incdec(0-200)' : "INC/DEC 0 to 200 days SMA increasing order (+5) and decreasing order    (-5 )",
                                "strat_incdec(0-100)" : "INC/DEC 0 to 100 days SMA increasing order (+4) and decreasing order (-4 )",
                                "strat_incdec(8-21)": "INC/DEC 8, 14, 21  days SMA increasing order (+3) and decreasing order (-3 )",
                                "strat_incdec(50-200)": "INC/DEC 50, 100, 200  days SMA increasing order (+1) and decreasing order (-1 )",
                                "strat_smaema" : "SMA/EMA CrossOvers",
                                'strat_point5all(near_close)': "POINT5 SMA are near 0.5% of close (+2)",
                                "strat_point5all(above_close)":"POINT5 SMA are in  0.5% of of each other but above close (-1)",
                                "strat_point5all(below_close)" : "POINT5 SMA are in  0.5% of of each other but above close (+1)",
                                "strat_macd(normal_crossover)" : "MACD change : positve gradient to negative gardient (+2) and vice versa (-2) ",                               
                                'strat_macd(middle_crossover)': "MACD crosees ZERO line : positive (+3) and negative   (-3) ",
                                'strat_macd(hist_turnover)': "MACD Histog. change: positve gradient to negative gardient (+1) and vice versa (-1)  ",                                
                                'strat_hieken_ashi(Bearish)': "HELKIN ASHI Negative candle :  if 4% or 2% of CL candle body the -3 or -2",                                
                                'strat_fisher(Bearish)': "FISHER Overbought zone cross over : positive crossover (+1) and negative cross over (-3)",
                                'strat_fisher(Bullish)': "FISHER Oversold zone cross over : positive crossover (+3) and negative cross over (-1)",
                                'strat_fisher(Middle_cross)': 'FISHER Middle zone cross over : positive crossover (+2) and negative cross over (-2)',
                                'strat_stochrsi(Bullish)': 'SRSI Oversold zone cross over : positive crossover (+2) and negative cross over (-1)',
                                'strat_stochrsi(Bearish)': 'SRSI Overbought zone cross over : positive crossover (+1) and negative cross over (-2)',
                                'strat_stochrsi(Middle_cross)': 'SRSI Middle zone cross over : positive crossover (+1) and negative cross over (-1)',                        
                                'strat_hieken_ashi(Bullish)': "HELKIN ASHI Positive candle helkinashi :  if 4% or 2% of CL candle body the +3 or +2",
                                'strat_hieken_ashi(Breakpoint)': 'HELKIN ASHI Change over hieken_ashi candle : (hand and leg shall be 1% of candle body (+4)',
                               }) 
cols = [' Net Score: High :  Bullish Low: Bearish midlle : Hold ', 'Sector', 'Close', 'PctChange', 'OverBuySell','Score', 'REMARKS', 'BREAKPOINTS',  'Pos_Score', 'Neg_Score', 'Pos_Strat', 'Neg_Strat', 'INC/DEC 0 to 200 days SMA increasing order (+5) and decreasing order    (-5 )', 'INC/DEC 0 to 100 days SMA increasing order (+4) and decreasing order (-4 )', 'INC/DEC 8, 14, 21  days SMA increasing order (+3) and decreasing order (-3 )', 'INC/DEC 50, 100, 200  days SMA increasing order (+1) and decreasing order (-1 )', 'SMA/EMA CrossOvers', 'POINT5 SMA are in  0.5% of of each other but above close (-1)', 'POINT5 SMA are in  0.5% of of each other but above close (+1)', 'MACD change : positve gradient to negative gardient (+2) and vice versa (-2) ', 'MACD crosees ZERO line : positive (+3) and negative   (-3) ', 'MACD Histog. change: positve gradient to negative gardient (+1) and vice versa (-1)  ', 'HELKIN ASHI Positive candle helkinashi :  if 4% or 2% of CL candle body the +3 or +2', 'HELKIN ASHI Negative candle :  if 4% or 2% of CL candle body the -3 or -2', 'FISHER Overbought zone cross over : positive crossover (+1) and negative cross over (-3)', 'FISHER Oversold zone cross over : positive crossover (+3) and negative cross over (-1)', 'FISHER Middle zone cross over : positive crossover (+2) and negative cross over (-2)', 'SRSI Oversold zone cross over : positive crossover (+2) and negative cross over (-1)', 'SRSI Overbought zone cross over : positive crossover (+1) and negative cross over (-2)', 'SRSI Middle zone cross over : positive crossover (+1) and negative cross over (-1)']
table = table[cols]
cols = ['Pos_Score', 'Neg_Score', 'Pos_Strat', 'Neg_Strat', 'INC/DEC 0 to 200 days SMA increasing order (+5) and decreasing order    (-5 )', 'INC/DEC 0 to 100 days SMA increasing order (+4) and decreasing order (-4 )', 'INC/DEC 8, 14, 21  days SMA increasing order (+3) and decreasing order (-3 )', 'INC/DEC 50, 100, 200  days SMA increasing order (+1) and decreasing order (-1 )', 'SMA/EMA CrossOvers', 'POINT5 SMA are in  0.5% of of each other but above close (-1)', 'POINT5 SMA are in  0.5% of of each other but above close (+1)', 'MACD change : positve gradient to negative gardient (+2) and vice versa (-2) ', 'MACD crosees ZERO line : positive (+3) and negative   (-3) ', 'MACD Histog. change: positve gradient to negative gardient (+1) and vice versa (-1)  ', 'HELKIN ASHI Positive candle helkinashi :  if 4% or 2% of CL candle body the +3 or +2', 'HELKIN ASHI Negative candle :  if 4% or 2% of CL candle body the -3 or -2', 'FISHER Overbought zone cross over : positive crossover (+1) and negative cross over (-3)', 'FISHER Oversold zone cross over : positive crossover (+3) and negative cross over (-1)', 'FISHER Middle zone cross over : positive crossover (+2) and negative cross over (-2)', 'SRSI Oversold zone cross over : positive crossover (+2) and negative cross over (-1)', 'SRSI Overbought zone cross over : positive crossover (+1) and negative cross over (-2)', 'SRSI Middle zone cross over : positive crossover (+1) and negative cross over (-1)']
for col in cols:
    table.loc[table[col] == 0, [col]] = ' '
date = str(dt.date.today())
if not os.path.exists('Results'):
        os.makedirs('Results')
else:
    if not os.path.exists('Results/{}'.format(date)):
        os.makedirs('Results/{}'.format(date))
    #name = "Results_" + str(dt.date.today())+ ".csv"
    name = "Results/" + ondate + "/all_stocks.csv"
    table.to_csv(name)

In [ ]:
cols = ['INC/DEC 0 to 200 days SMA increasing order (+5) and decreasing order    (-5 )', 'INC/DEC 0 to 100 days SMA increasing order (+4) and decreasing order (-4 )', 'INC/DEC 8, 14, 21  days SMA increasing order (+3) and decreasing order (-3 )', 'INC/DEC 50, 100, 200  days SMA increasing order (+1) and decreasing order (-1 )', 'SMA/EMA CrossOvers', 'POINT5 SMA are in  0.5% of of each other but above close (-1)', 'POINT5 SMA are in  0.5% of of each other but above close (+1)', 'MACD change : positve gradient to negative gardient (+2) and vice versa (-2) ', 'MACD crosees ZERO line : positive (+3) and negative   (-3) ', 'MACD Histog. change: positve gradient to negative gardient (+1) and vice versa (-1)  ', 'HELKIN ASHI Positive candle helkinashi :  if 4% or 2% of CL candle body the +3 or +2', 'HELKIN ASHI Negative candle :  if 4% or 2% of CL candle body the -3 or -2', 'FISHER Overbought zone cross over : positive crossover (+1) and negative cross over (-3)', 'FISHER Oversold zone cross over : positive crossover (+3) and negative cross over (-1)', 'FISHER Middle zone cross over : positive crossover (+2) and negative cross over (-2)', 'SRSI Oversold zone cross over : positive crossover (+2) and negative cross over (-1)', 'SRSI Overbought zone cross over : positive crossover (+1) and negative cross over (-2)', 'SRSI Middle zone cross over : positive crossover (+1) and negative cross over (-1)']
for col in cols:
    table.loc[table[col] == 0, [col]] = ' '

In [ ]:
all_sectors = table.Sector.unique()

for s in all_sectors:
    Sectorial_table = table[table.Sector == s]
    name = "Sector_" + s +"_"+ str(ondate)+ ".csv"
    Sectorial_table.to_csv('Results/{}/{}'.format(date,name))    
    

In [ ]:
display(complete_results['HDFCBANK'])
print(strat_stochrsi_results)

In [ ]:
df = pd.read_csv('stocks_dfs/{}.csv'.format('CONCOR'), header= 0)
exp1 = df['Close'].ewm(span=12, adjust=False).mean()
exp2 = df['Close'].ewm(span=26, adjust=False).mean()
macd = exp1-exp2
signal = macd.ewm(span=9, adjust=False).mean()
df['hist'] = macd-signal
hist = TA.SMA(df, 3, "hist")
print(df['hist'])
print(hist)


In [ ]:
pctchange = {'Company': [],'Sector':[], 'feb': [], 'march':[], 'dip': [], 'today':[], 'remaining':[], 'recovered':[], 'order' : [] }
import math
for ticker in tickers:
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    df.index = df.Date 
    if df.index[-1] != str(dt.date.today()):
        continue
    if '2020-02-24'in df.index:
        if '2020-03-24' in df.index:
            inside = 0 
            feb = df.loc['2020-02-24']['Close']
            march = df.loc['2020-03-24']['Close']
            today = df.iloc[-1]['Close']
            if march > feb :
                dip = ((march-feb)/feb)*100
                recovered = 0
                remaining = 0
                inside = 1
            elif today > feb:
                dip = ((march-feb)/feb)*100
                recovered = 100+((today-feb)/feb)*100
                remaining = 0
                inside = 1
            elif march < today < feb:
                dip = ((march-feb)/feb)*100
                recovered = ((today-march)/feb)*100
                remaining = ((today-feb)/feb)*100
                inside = 1
            else :
                dip = ((march-feb)/feb)*100
                recovered = 0
                remaining = ((today - feb)/feb)*100 -100
                inside = 1
                
            if inside == 0:
                print(ticker)
            pctchange['Company'].append(ticker)
            if ticker == 'NIFTY':
                pctchange['Sector'].append('-')
            else :
                pctchange['Sector'].append(Sector[ticker])
            pctchange['dip'].append(math.ceil(dip))
            pctchange['today'].append(math.ceil(today))
            pctchange['feb'].append(math.ceil(feb))            
            pctchange['march'].append(math.ceil(march))            
            pctchange['recovered'].append(math.ceil(recovered))
            pctchange['remaining'].append(math.ceil(remaining))  
            pctchange['order'].append(math.ceil(remaining * dip))  

            #print(feb, march, today, dip, recovered, remaining)  
    
       


In [ ]:
for i in pctchange:
    print(i, len(pctchange[i]))
table = pd.DataFrame.from_dict(pctchange)
table = table.set_index('Company')    
table = table.sort_values(by=['Sector', 'order', 'recovered'], ascending=False)
table.to_csv('pctchange.csv') 
print(str(dt.date.today()))
display(table)

In [ ]:
def new_close(df):
    
    newclose = (14*4*df.iloc[-1]['14ma'] - 4*df.iloc[-14]['Close'] - 8*7*df.iloc[-1]['8ma'] + 7*df.iloc[-8]['Close'])/3
    new8ma= (df.iloc[-1]['8ma']*8 - df.iloc[-8]['Close'] + newclose)/8
    new14ma= (df.iloc[-1]['14ma']*14 - df.iloc[-14]['Close'] + newclose)/14
    new21ma= (df.iloc[-1]['21ma']*21 - df.iloc[-21]['Close'] + newclose)/21
    pctchange = (newclose - df.iloc[-1]['Close'])*100/df.iloc[-1]['Close']
    if -5 < pctchange < 5:
        #print(df.iloc[-1]['8ma'], df.iloc[-1]['14ma'] , df.iloc[-1]['21ma'])
        #print(new8ma, new14ma, new21ma )
        if new14ma > new21ma :
            return newclose, pctchange, "Bullish" 
        else:
            return newclose, pctchange, "Bearish" 
    else:
        return 0, 0 , ""

def prediction_inc_dec(ticker):
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    if (df.iloc[-1]['14ma'] > df.iloc[-1]['8ma'] > df.iloc[-1]['21ma']) or (df.iloc[-1]['21ma'] > df.iloc[-1]['8ma']> df.iloc[-1]['14ma']):
        newclose, pctchange ,bb= new_close(df)
        if newclose != 0 and newclose > df.iloc[-1]['Close'] :
            print(ticker , "8-14", bb," Cross at ", round(newclose,2), "with" , round(pctchange,2) , "%")
            #company_results[ticker]['Prediction_crossover'] = "8-14 Bullish Cross at " + newclose        
        if newclose != 0 and newclose < df.iloc[-1]['Close'] :
            print(ticker , "8-14", bb, " Cross at ", round(newclose,2), "with" , round(pctchange,2) , "%")
            #company_results[ticker]['Prediction_crossover'] = "8-14 Bearish Cross at " + newclose        
        

for ticker in tickers:
    prediction_inc_dec(ticker)
    
    

In [47]:
def new_close(df, curr_seq, next_seq):
    seq = {'8ma': 1 , '14ma': 2, '21ma':3}
    if curr_seq[0] == next_seq[0]:
        ma1 = curr_seq[1]
        ma2 = curr_seq[2]
    elif curr_seq[1] == next_seq[1]:
        ma1 = curr_seq[0]
        ma2 = curr_seq[2]
    elif curr_seq[2] == next_seq[2]:
        ma1 = curr_seq[0]
        ma2 = curr_seq[1]
    else:
        ma1 = "8ma"
        ma2 = "14ma"
        
    if seq[ma1]<seq[ma2]:
        bb = "BEARISH"
    if seq[ma1]>seq[ma2]:
        bb = "BULLISH" 
        
    if (ma1 == "8ma" and ma2 == "14ma") or (ma1 == "14ma" and ma2 == "8ma"):
        newclose = (14*4*df.iloc[-1]['14ma'] - 4*df.iloc[-14]['Close'] - 8*7*df.iloc[-1]['8ma'] + 7*df.iloc[-8]['Close'])/3
    if (ma1 == "14ma" and ma2 == "21ma") or (ma1 == "21ma" and ma2 == "14ma"):
        newclose = (14*21*df.iloc[-1]['21ma'] - 14*df.iloc[-21]['Close'] - 14*21*df.iloc[-1]['14ma'] + 21*df.iloc[-14]['Close'])/7
    if (ma1 == "8ma" and ma2 == "21ma") or (ma1 == "21ma" and ma2 == "8ma"):
        newclose = (8*21*df.iloc[-1]['21ma'] - 8*df.iloc[-21]['Close'] - 8*21*df.iloc[-1]['8ma'] + 21*df.iloc[-8]['Close'])/13
        
    pctchange = (newclose - df.iloc[-1]['Close'])*100/df.iloc[-1]['Close']
    #print(ma1, ma2)
    return bb, newclose, pctchange , ma1, ma2
    

def prediction_inc_dec(ticker):
    global inc_count
    global dec_count
    df = pd.read_csv('stocks_dfs/{}.csv'.format(ticker), header= 0)
    closes = [df.iloc[-1]['Close'], df.iloc[-1]['Close']*1.05, df.iloc[-1]['Close']*0.95] 
    for close in closes:
        new8ma= (df.iloc[-1]['8ma']*8 - df.iloc[-8]['Close'] + close)/8
        new14ma= (df.iloc[-1]['14ma']*14 - df.iloc[-14]['Close'] + close)/14
        new21ma= (df.iloc[-1]['21ma']*21 - df.iloc[-21]['Close'] + close)/21
        mas_values = [df.iloc[-1]['8ma'] , df.iloc[-1]['14ma'] , df.iloc[-1]['21ma']]
        newmas_values = [new8ma, new14ma, new21ma]
        mas_values = sorted(mas_values, reverse = True)
        newmas_values = sorted(newmas_values, reverse = True)
        mas = ['8ma', '14ma', '21ma']
        curr_seq = []
        next_seq = []
        for x in mas_values:
            if x == df.iloc[-1]['8ma'] :
                curr_seq.append("8ma")
            if x == df.iloc[-1]['14ma'] :
                curr_seq.append("14ma")
            if x == df.iloc[-1]['21ma'] :
                curr_seq.append("21ma")

        for x in newmas_values:
            if x == new8ma :
                next_seq.append("8ma")
            if x == new14ma :
                next_seq.append("14ma")
            if x == new21ma :
                next_seq.append("21ma")
                
        if curr_seq != next_seq:
            if close == df.iloc[-1]['Close']:
                bb , newclose, pctchange,ma1, ma2 = new_close(df, curr_seq, next_seq)
                #print(curr_seq, next_seq)
                print(ticker , bb , ma1 , ma2 , " Cross at ", round(close,2), "with" , round(0,2) , "%")
                return
            else:
                inc_count+= 1
                bb , newclose, pctchange,ma1, ma2 = new_close(df, curr_seq, next_seq)
                #print(curr_seq, next_seq)
                print(ticker , bb , ma1 , ma2 , " Cross at ", round(newclose,2), "with" , round(pctchange,2) , "%")
    
inc_count = 0
dec_count = 0
for ticker in tickers:
    prediction_inc_dec(ticker)
    
    

    
    

3MINDIA BULLISH 21ma 8ma  Cross at  21947.09 with 0.9 %
ACC BEARISH 8ma 14ma  Cross at  1344.88 with -4.79 %
AUBANK BEARISH 8ma 21ma  Cross at  696.95 with 0 %
ADANIGREEN BULLISH 21ma 14ma  Cross at  360.65 with 0 %
ADANITRANS BULLISH 21ma 8ma  Cross at  249.4 with 0 %
ABCAPITAL BULLISH 21ma 8ma  Cross at  59.05 with 0 %
ABFRL BEARISH 8ma 21ma  Cross at  114.73 with -2.77 %
ASHOKLEY BULLISH 14ma 8ma  Cross at  52.2 with 0 %
ASTERDM BULLISH 14ma 8ma  Cross at  140.82 with 3.66 %
ASTERDM BEARISH 8ma 21ma  Cross at  130.63 with -3.84 %
ASTRAZEN BULLISH 21ma 8ma  Cross at  3387.04 with 1.63 %
ASTRAZEN BEARISH 8ma 14ma  Cross at  3176.9 with -4.68 %
AXISBANK BULLISH 14ma 8ma  Cross at  456.62 with 1.92 %
BEML BULLISH 14ma 8ma  Cross at  673.9 with 0 %
BAJAJFINSV BULLISH 21ma 8ma  Cross at  6440.35 with 0 %
BAJAJHLDNG BULLISH 21ma 8ma  Cross at  2725.65 with 0 %
BANKBARODA BULLISH 14ma 8ma  Cross at  47.1 with 0 %
BANKINDIA BEARISH 14ma 21ma  Cross at  46.75 with -2.91 %
BPCL BEARISH 14ma 21

WABCOINDIA BEARISH 8ma 14ma  Cross at  6660.62 with -4.88 %
ZEEL BULLISH 14ma 8ma  Cross at  159.75 with 0 %
